In [13]:
# Imports
import pandas as pd
import numpy as np
from numpy.random import randint as some_int
from numpy.random import rand as some_float

In [11]:
print("\t".join(['1','2','3']))

1	2	3


In [16]:
for i in range(10):
    print(some_int(1,100))

75
67
44
54
65
17
64
16
1
81


In [75]:
def show_table(table):
    print("User_id \tNum_shares \tPrice\n")
    for entry in table:
        print("\t\t".join([str(i) for i in entry]))

## Base Logic for concurrency and trade execution
The above state is what remains at any given time in our database.<br>
When a new request comes (bid or ask), then we add that entry to the table if it is not the highest bid/ lowest ask.<br>
If it is the highest bid, then we check to see if it is >= lowest ask, <br>
&nbsp; &nbsp; &nbsp; &nbsp; If it is, we do the transaction (exchange shares and money)<br><br>

If it is the lowest ask, then we check to see if it is <= highest bid, <br>
&nbsp; &nbsp; &nbsp; &nbsp; If it is, we do the transaction (exchange shares and money)<br><br>

Finally, we update our tables to get back into the state where highest bid < lowest ask.<br><br>

The lowest ask is what we show as the price of the stock for anyone looking to buy.<br>
The highest bid is what we show as the price of the stock for anyone looking to sell.

In [110]:
'''
This is the current state space:
Every stock under consideration (xyz) will have 2 lists:
1. Bid requests (offer to buy n shares of xyz stock at p price) -> arranged in descending order
2. Ask requests (offer to sell up to n shares of xyz stock at p price) -> arranged in ascending order
'''

class Bid_Ask:
    def __init__(self):
        
        # Initialize both tables with a blank and then fill in some initial values
        self.bids = []
        self.asks = []

        # The price below is arbitrary, I am adding it to get a table more resembling the real one
        lowest_bid = 50
        bid_spread = int(lowest_bid*0.1)

        for bid in range(some_int(5,10)):
            user_id = 'User' + str(some_int(1,20))
            num_shares = some_int(10,30)
            bid_price = lowest_bid + bid_spread*some_float()

            self.bids.append([user_id, num_shares, bid_price])
        self.re_sort('bids')

        # The price below is arbitrary, I am adding it to get a table more resembling the real one
        lowest_ask = int(max([x[2] for x in self.bids]))+1
        ask_spread = int(lowest_ask*0.1)

        for ask in range(some_int(5,10)):
            user_id = 'User' + str(some_int(11,30))
            num_shares = some_int(10,30)
            ask_price = lowest_ask + ask_spread*some_float()

            self.asks.append([user_id, num_shares, ask_price])

        self.re_sort('asks')
        
        self.entry = []

    def re_sort(self,table_name):
        if table_name == 'bids':
            self.bids = sorted(self.bids, key = lambda x: -x[2])
        else:
            self.asks = sorted(self.asks, key = lambda x: x[2])
            
    def show_table(self, table):
        '''
        table is either a bid table or an ask table.
        The shape is (number_of_entries, 3):
            The 3 columns are User_id, num_shares and price
        '''
        print("User_id \t\tNum_shares \t\tPrice\n\n")
        for entry in table:
            print("\t\t".join([str(i) for i in entry]))

    def show_bids(self):
        print('-'*50)
        print('Bid table sorted in descending order:\n')
        show_table(self.bids)
        print('-'*50)
    
    def show_asks(self):
        print('-'*50)
        print('Ask table sorted in ascending order:\n')
        show_table(self.asks)
        print('-'*50)

    def handle_bid_entry(self):
        
        transaction = {}
        
        lowest_ask = self.asks[0]

        bid_price = self.entry[2]
        lowest_ask_price = lowest_ask[2]

        bid_shares = self.entry[1]
        ask_shares = lowest_ask[1]

        if bid_price >= lowest_ask_price:

            transaction['buyer'] = self.entry[0]
            transaction['seller'] = lowest_ask[0]
            transaction['price'] = lowest_ask_price

            remaining_shares = ask_shares - bid_shares
            if remaining_shares <= 0:
                # If the number of shares to buy > number of shares to sell, then all the available shares will be sold and the bid entry will be updated with a reduced number of bid_shares
                transaction['num_shares'] = ask_shares
                self.entry[1] = -remaining_shares

                # Delete the ask(seller) entry since the seller sold all the shares acc to the lowest ask request
                self.asks = self.asks[1:]
            else:
                transaction['num_shares'] = bid_shares
                self.entry[1] = 0
                self.asks[0][1] = remaining_shares
        else:
            # No transaction right now, just add the entry to the table
#             print(self.entry)
            self.bids.append(list(self.entry))
            self.re_sort('bids')
            self.entry[1] = 0
            
        return transaction
    
    def handle_ask_entry(self):
        
        transaction = {}
        
        highest_bid = self.bids[0]

        ask_price = self.entry[2]
        highest_bid_price = highest_bid[2]

        ask_shares = self.entry[1]
        bid_shares = highest_bid[1]

        if ask_price <= highest_bid_price:

            transaction['seller'] = self.entry[0]
            transaction['buyer'] = highest_bid[0]
            transaction['price'] = highest_bid_price

            remaining_shares = bid_shares - ask_shares
            if remaining_shares <= 0:
                # If the number of shares to sell > number of shares to buy, then all the available shares will be bought and the ask entry will be updated with a reduced number of ask_shares
                transaction['num_shares'] = bid_shares
                self.entry[1] = -remaining_shares

                # Delete the bid(buyer) entry since the buyer bought all the shares acc to the highest bid request
                self.asks = self.asks[1:]
            else:
                transaction['num_shares'] = ask_shares
                self.entry[1] = 0
                self.bids[0][1] = remaining_shares
        else:
            # No transaction right now, just add the entry to the table
#             print(self.entry)
            self.asks.append(list(self.entry))
            self.re_sort('asks')
            self.entry[1] = 0
        
        return transaction

        
        
    def add_entry(self, entry, type_of_entry = 'bid'):
        self.entry = entry
#         print('The entry to handle:\n', self.entry)
        if type_of_entry == 'bid':
            transaction = self.handle_bid_entry()
                    
        else:
            transaction = self.handle_ask_entry()
                    
        if transaction != {}:
            print("Transaction details:\n",transaction,'\n\n')
        if self.entry[1] != 0:
#             print(self.entry)
            # Say you are a buyer who wants to buy 150 shares for Rs. 100 and the lowest selling price is Rs. 90 but only for 50 shares
            # Then we want to execute 1 transaction of selling 50 shares and for Rs.90 and for the remaining 100,
            # We need to run the updated entry for buying 100 shares for Rs.100 now
            self.add_entry(entry, type_of_entry = type_of_entry)
        else:
            pass
                

    def __str__(self):
        print('Current Statespace includes:\n\nCurrent Bid table:\n')
        self.show_bids()
        print('\n\n\nCurrent Ask table:\n')
        self.show_asks()
        return ''



In [111]:
# We are creating an instance corresponding to a particular stock (eg. XYZ shell corporation)

xyz_stock_manager = Bid_Ask()
print(xyz_stock_manager)

Current Statespace includes:

Current Bid table:

--------------------------------------------------
Bid table sorted in descending order:

User_id 	Num_shares 	Price

User17		10		54.537348510427044
User1		26		53.9997830715096
User19		27		52.981319671315774
User7		17		52.957897100056115
User5		26		52.23564698933455
User8		12		51.915853262370476
User3		23		51.11372919866043
--------------------------------------------------



Current Ask table:

--------------------------------------------------
Ask table sorted in ascending order:

User_id 	Num_shares 	Price

User15		25		55.2005916483199
User18		27		57.13453432837325
User22		28		57.31495545775966
User25		28		57.54509321293834
User19		16		57.9915510337763
User13		18		59.01651138612718
User23		11		59.479939651673504
--------------------------------------------------



In [112]:
'''
Bid request, Number of shares to buy < number of shares to sell (for cheapest seller)
only 1 transaction will happen, no entries added to the table
'''
example_1 = Bid_Ask()
print(example_1)

user = 'Ansh'
num_shares = example_1.asks[0][1]-some_int(1,5)
price = (example_1.asks[0][2] + example_1.asks[1][2])/2.0
entry = [user, num_shares, price]
example_1.add_entry(entry, 'bid')
print(example_1)

Current Statespace includes:

Current Bid table:

--------------------------------------------------
Bid table sorted in descending order:

User_id 	Num_shares 	Price

User17		16		54.514537163634614
User6		27		53.52546481788915
User8		12		51.93421070915845
User9		28		51.51005162584786
User8		28		50.536186166113026
--------------------------------------------------



Current Ask table:

--------------------------------------------------
Ask table sorted in ascending order:

User_id 	Num_shares 	Price

User24		29		56.03434905830408
User26		28		56.29836923388798
User13		15		56.336927668113916
User11		29		56.99103229565214
User20		22		57.16627118536149
User22		29		58.11928345155473
User21		22		59.51426426345322
User13		16		59.723474359345175
--------------------------------------------------

Transaction details:
 {'buyer': 'Ansh', 'seller': 'User24', 'price': 56.03434905830408, 'num_shares': 26} 


Current Statespace includes:

Current Bid table:

----------------------------------------

In [113]:
'''
Bid request, Number of shares to buy = number of shares to sell (for cheapest seller)
only 1 transaction will happen, no entries added to the table, cheapest seller entry will be removed
'''
example_2 = Bid_Ask()
print(example_2)

user = 'Ansh'
num_shares = example_2.asks[0][1]
price = (example_2.asks[0][2] + example_2.asks[1][2])/2.0
entry = [user, num_shares, price]
example_2.add_entry(entry, 'bid')
print(example_2)

Current Statespace includes:

Current Bid table:

--------------------------------------------------
Bid table sorted in descending order:

User_id 	Num_shares 	Price

User10		25		54.61761918668042
User9		12		54.2587529673536
User2		27		53.75510690439164
User1		13		53.242127510967826
User19		15		50.97378755091176
User1		20		50.74055608106315
User2		10		50.40365280306979
--------------------------------------------------



Current Ask table:

--------------------------------------------------
Ask table sorted in ascending order:

User_id 	Num_shares 	Price

User20		16		55.56300970098196
User15		13		55.5850051202828
User29		20		57.10751409992616
User11		28		57.235894557730965
User11		26		58.12147121306802
User23		25		58.69345428318774
User16		15		59.41579475152797
--------------------------------------------------

Transaction details:
 {'buyer': 'Ansh', 'seller': 'User20', 'price': 55.56300970098196, 'num_shares': 16} 


Current Statespace includes:

Current Bid table:

---------------

In [ ]:
'''
Bid request, Number of shares to buy > number of shares to sell (for cheapest seller)
only 1 transaction will happen, no entries added to the table
'''
example_1 = Bid_Ask()
print(example_1)

user = 'Ansh'
num_shares = example_1.asks[0][1]-some_int(1,5)
price = (example_1.asks[0][2] + example_1.asks[1][2])/2.0
entry = [user, num_shares, price]
example_1.add_entry(entry, 'bid')
print(example_1)

In [60]:
'''
Now, I am buying 60 more shares at a price up to 56.5
There is a seller (User 28) who is willing to sell up to 32 shares for ~ 55.25
Here, only 1 transaction will be processed as my entry (buy request from Ansh) is completed
'''
user = 'Ansh'
num_shares = 60
price = 57
entry = [user, num_shares, price]
xyz_stock_manager.add_entry(entry, 'bid')
print(xyz_stock_manager)

Transaction details:
 {'buyer': 'Ansh', 'seller': 'User21', 'price': 55.350408215707546, 'num_shares': 57} 


['Ansh', 3, 57]
Transaction details:
 {'buyer': 'Ansh', 'seller': 'User18', 'price': 56.74815121746484, 'num_shares': 3} 


Current Statespace includes:

Current Bid table:

User_id 		Num_shares 		Price

User3		23		54.429077996206104
User2		68		53.19548620704463
User6		7		52.26451313771323
User9		99		51.69539714213477
User17		25		51.539832296918696
User15		13		50.40202185230004



Current Ask table:

User_id 		Num_shares 		Price

User18		67		56.74815121746484
User28		13		56.892405852157694
User22		96		57.59420671900062
User15		28		57.82016103506491
User28		20		57.9893522662901
User16		74		58.04799369409673
User22		20		59.24503714479006
User21		53		59.24586477646837

